In [1]:
%pip install pandas
%pip install firebase-admin

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Fungsi untuk ekstraksi data dari CSV, transformasi, dan upload ke Firebase Storage
def process_and_upload_data(csv_file):
    try:
        # Ekstraksi data dari CSV
        df = pd.read_csv(csv_file)

        # Transformasi data
        df_cleaned = df.drop_duplicates().dropna()
        df_cleaned = df_cleaned[df_cleaned['age'] != "-"]
        df_cleaned = df_cleaned[df_cleaned['favorite_front_end'] != "-"]
        
        # Simpan data yang telah ditransformasi ke CSV
        cleaned_csv = "survey_cleaned.csv"
        df_cleaned.to_csv(cleaned_csv, index=False)
        
        # Load data yang telah ditransformasi ke Firebase Storage
        upload_to_firebase(cleaned_csv)
        
        print("Proses berhasil.")
    except Exception as e:
        print("Error:", e)

# Fungsi untuk upload data ke Firebase Storage
def upload_to_firebase(csv_file):
    import firebase_admin
    from firebase_admin import credentials, storage

    try:
        cred = credentials.Certificate("keyErista.json")
        firebase_admin.initialize_app(cred, {
            "storageBucket": "erista-cloud.appspot.com"
        })

        bucket = storage.bucket()
        blob = bucket.blob(csv_file)
        blob.upload_from_filename(csv_file, content_type='text/csv')

        print("Upload ke Firebase Storage berhasil.")
    except Exception as e:
        print("Error:", e)

# Jalankan fungsi utama
if __name__ == "__main__":
    process_and_upload_data("survey.csv")

Upload ke Firebase Storage berhasil.
Proses berhasil.


In [7]:
import pandas as pd
import requests
from io import StringIO

# Fungsi untuk ekstraksi data dari JSON URL, transformasi, dan upload ke Firebase Storage
def process_and_upload_data(json_url, filename):
    try:
        # Ekstraksi data dari JSON URL
        response = requests.get(json_url)
        data = response.json()
        df = pd.DataFrame(data)
        
        # Transformasi data
        df_transformed = df[(df['payment_method'] == 'credit card') & (df['status'] == 'success')]
        
        # Simpan DataFrame yang telah ditransformasi ke dalam string CSV
        csv_data = df_transformed.to_csv(index=False)
        
        # Upload data yang telah ditransformasi ke Firebase Storage
        upload_to_firebase(csv_data, filename)
        
        print("Proses berhasil.")
    except Exception as e:
        print("Error:", e)

# Fungsi untuk upload data ke Firebase Storage
def upload_to_firebase(csv_data, filename):
    import firebase_admin
    from firebase_admin import credentials, storage

    try:
        if not firebase_admin._apps:
            cred = credentials.Certificate("keyErista.json")
            firebase_admin.initialize_app(cred, {
                "storageBucket": "erista-cloud.appspot.com"
            })

        bucket = storage.bucket()
        blob = bucket.blob(filename)
        blob.upload_from_string(csv_data, content_type='text/csv')

        print("Upload ke Firebase Storage berhasil.")
    except Exception as e:
        print("Error:", e)

# Jalankan fungsi utama
if __name__ == "__main__":
    process_and_upload_data(
        json_url="https://gist.githubusercontent.com/nadirbslmh/8fc9cc6cd5cbaaf5cbff63b090fb497e/raw/a7bf3e1edab88b04314a40a9de3ed744bc86d0e9/ecommerce.json",
        filename="transaksi.csv"
    )

Upload ke Firebase Storage berhasil.
Proses berhasil.
